In [2]:
import sqlite3
import pandas as pd
%matplotlib inline

In [1]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english

In [2]:
def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

In [221]:
#dbname='wrc18_sweden.db'
dbname='wrc18.db'
conn = sqlite3.connect(dbname)

q="SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(q,conn)

,name
0,itinerary_event
1,itinerary_legs
2,itinerary_sections
3,itinerary_stages
4,itinerary_controls
5,startlists
6,startlist_classes
7,penalties
8,retirements
9,stagewinners


In [222]:
rally='Sweden'

In [641]:
q='''
SELECT * FROM championship_events ce
WHERE ce.`country.name`="{rally}"
'''.format(rally=rally)
pd.read_sql(q,conn)

,categories,clerkOfTheCourse,country.countryId,country.iso2,country.iso3,country.name,countryId,eventId,finishDate,location,...,organiserUrl,slug,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId
0,None,None,215,SE,SWE,Sweden,215,27,2018-02-18,Sweden,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587


In [384]:
#Get list of itinerary sections for a given rally
q='''
SELECT il.legDate, il.name AS date, il.startListId, il.status, isc.* FROM championship_events ce 
JOIN itinerary_event ie ON ce.eventId = ie.eventId 
JOIN itinerary_legs il ON ie.itineraryId=il.itineraryId
JOIN itinerary_sections isc ON il.itineraryLegId=isc.itineraryLegId
WHERE ce.`country.name`="{rally}"
'''.format(rally=rally)
itinerary_sections = pd.read_sql(q,conn)
itinerary_sections

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,name,order
0,2018-02-15,Thursday 15 February,74,Completed,53,117,Section 1,1
1,2018-02-16,Friday 16 February,78,Completed,54,118,Section 2,2
2,2018-02-16,Friday 16 February,78,Completed,54,119,Section 3,3
3,2018-02-17,Saturday 17 February,84,Completed,55,120,Section 4,4
4,2018-02-17,Saturday 17 February,84,Completed,55,121,Section 5,5
5,2018-02-17,Saturday 17 February,84,Completed,55,122,Section 6,6
6,2018-02-18,Sunday 18 February,88,Completed,56,123,Section 7,7
7,2018-02-18,Sunday 18 February,88,Completed,56,124,Section 8,8
8,2018-02-18,Sunday 18 February,88,Completed,56,125,Section 9,9


In [404]:
rules = {'origin': "#date# #name# "}

In [405]:
grouped = itinerary_sections.sort_values('order').groupby('date',sort=False)

for name, group in grouped:
    print('---')
    for r in group.apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1):
        print(r)

---
Thursday 15 February Section 1 
---
Friday 16 February Section 2 
Friday 16 February Section 3 
---
Saturday 17 February Section 4 
Saturday 17 February Section 5 
Saturday 17 February Section 6 
---
Sunday 18 February Section 7 
Sunday 18 February Section 8 
Sunday 18 February Section 9 


In [402]:
#Get list of itinerary stages for a given rally
q='''SELECT * FROM itinerary_stages ist
JOIN championship_events ce ON ist.eventId=ce.eventId
WHERE ce.`country.name`="{rally}" ORDER BY number ASC
'''.format(rally=rally)
pd.read_sql(q,conn).head(3)

,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,...,organiserUrl,slug,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId
0,SS1,1.90,27,SS1 SSS Karlstad 1,1,317,HeadToHeadSuperSpecialStage,Completed,Tenth,53,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587
1,SS2,21.26,27,SS2 Hof-Finnskog 1,2,299,SpecialStage,Completed,Tenth,54,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587
2,SS3,24.88,27,SS3 Svullrya 1,3,300,SpecialStage,Completed,Tenth,54,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587


In [409]:
#Get annotated stages of a particular rally
q='''
SELECT il.legDate, il.name AS date, il.startListId, il.status,
    isc.itineraryLegId, isc.itinerarySectionId, isc.name AS section, isc.`order`,
    ist.* FROM championship_events ce 
JOIN itinerary_event ie ON ce.eventId = ie.eventId 
JOIN itinerary_legs il ON ie.itineraryId=il.itineraryId
JOIN itinerary_sections isc ON il.itineraryLegId=isc.itineraryLegId
JOIN itinerary_stages ist ON ist.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
WHERE ce.`country.name`="{rally}" ORDER BY isc.`order`
'''.format(rally=rally)
rally_stages = pd.read_sql(q,conn)
rally_stages.head(3)

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-02-15,Thursday 15 February,74,Completed,53,117,Section 1,1,SS1,1.90,27,SS1 SSS Karlstad 1,1,317,HeadToHeadSuperSpecialStage,Completed,Tenth,53,117
1,2018-02-16,Friday 16 February,78,Completed,54,118,Section 2,2,SS2,21.26,27,SS2 Hof-Finnskog 1,2,299,SpecialStage,Completed,Tenth,54,118
2,2018-02-16,Friday 16 February,78,Completed,54,118,Section 2,2,SS3,24.88,27,SS3 Svullrya 1,3,300,SpecialStage,Completed,Tenth,54,118


In [562]:
rules = {'origin': "#name# (#distance# km)",
         'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'car':"#manufacturername# #vehicleModel#",
        'entrant':"#entrantname#"}

In [563]:
grouped = rally_stages.sort_values('order').groupby('date',sort=False)
for key in grouped.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=grouped.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        print('{}\n'.format(key2))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Thursday 15 February:

Section 1

	SS1 SSS Karlstad 1 (1.9 km)


---

Friday 16 February:

Section 2

	SS2 Hof-Finnskog 1 (21.26 km)
	SS3 Svullrya 1 (24.88 km)
	SS4 Röjden 1 (19.13 km)


Section 3

	SS5 Hof-Finnskog 2 (21.26 km)
	SS6 Svullrya 2 (24.88 km)
	SS7 Röjden 2 (19.13 km)
	SS8 Torsby 1 (9.56 km)


---

Saturday 17 February:

Section 4

	SS10 Hagfors 1 (23.4 km)
	SS11 Vargåsen 1 (14.21 km)
	SS9 Torntorp 1 (19.88 km)


Section 5

	SS12 Torntorp 2 (19.88 km)
	SS13 Hagfors 2 (23.4 km)
	SS14 Vargåsen 2 (14.21 km)


Section 6

	SS15 SSS Karlstad 2 (1.9 km)
	SS16 Torsby Sprint (3.43 km)


---

Sunday 18 February:

Section 7

	SS17 Likenas 1 (21.19 km)


Section 8

	SS18 Likenas 2 (21.19 km)


Section 9

	SS19 Torsby 2 (9.56 km)




In [249]:
section='Section 4'

In [250]:
#Get stages in a particular section of a particular rally
q='''
SELECT il.legDate, il.name AS date, il.startListId, il.status, isc.*, ist.* FROM championship_events ce 
JOIN itinerary_event ie ON ce.eventId = ie.eventId 
JOIN itinerary_legs il ON ie.itineraryId=il.itineraryId
JOIN itinerary_sections isc ON il.itineraryLegId=isc.itineraryLegId
JOIN itinerary_stages ist ON ist.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
WHERE ce.`country.name`="{rally}"  AND isc.name="{section}"
'''.format(rally=rally,section=section)
pd.read_sql(q,conn)

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,name,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-02-17,Saturday 17 February,84,Completed,55,120,Section 4,4,SS9,19.88,27,SS9 Torntorp 1,9,306,SpecialStage,Completed,Tenth,55,120
1,2018-02-17,Saturday 17 February,84,Completed,55,120,Section 4,4,SS11,14.21,27,SS11 Vargåsen 1,11,307,SpecialStage,Completed,Tenth,55,120
2,2018-02-17,Saturday 17 February,84,Completed,55,120,Section 4,4,SS10,23.40,27,SS10 Hagfors 1,10,316,SpecialStage,Completed,Tenth,55,120


In [229]:
code='SS12'

q='''SELECT * FROM itinerary_stages ist
JOIN championship_events ce ON ist.eventId=ce.eventId
WHERE ce.`country.name`="{rally}" AND ist.code="{code}"
'''.format(rally=rally, code=code)
pd.read_sql(q,conn)

,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,...,organiserUrl,slug,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId
0,SS12,19.88,27,SS12 Torntorp 2,12,308,SpecialStage,Completed,Tenth,55,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587


In [474]:
q='''
SELECT itc.*,ce.*
FROM itinerary_controls itc
JOIN championship_events ce ON itc.eventId=ce.eventId
WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL  ORDER BY firstCarDueDateTimeLocal 
'''.format(rally=rally)
pd.read_sql(q,conn).head()

,code,controlId,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,targetDuration,...,organiserUrl,slug,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId
0,TC1,1249,NaN,27,2018-02-15T19:04:00,2018-02-15T20:04:00+01:00,Start Farjestad Trotting Course Karlstad,317,Completed,00:00:00,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587
1,SS1,1251,1.90,27,2018-02-15T19:08:00,2018-02-15T20:08:00+01:00,Super Special Stage Karlstad 1,317,Completed,00:04:00,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587
2,TC1A,1194,102.33,27,2018-02-15T20:54:00,2018-02-15T21:54:00+01:00,Parc Ferme Torsby IN,-1,Completed,01:46:00,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587
3,TC1B,1193,NaN,27,2018-02-16T05:00:00,2018-02-16T06:00:00+01:00,Parc Ferme Torsby OUT/Service IN,-1,Completed,00:00:00,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587
4,TC1C,1195,NaN,27,2018-02-16T05:15:00,2018-02-16T06:15:00+01:00,Service OUT,-1,Completed,00:15:00,...,,rally-sweden-2018,2018-02-15,None,"Snow, Ice",rally-sweden.dotx,W. Europe Standard Time,"(UTC+01:00) Amsterdam, Berlin, Bern, Rome, Sto...",60,2587


In [ ]:
['categories', 'clerkOfTheCourse', 'country.countryId', 'country.iso2',
       'country.iso3', 'country.name', 'countryId', 'eventId', 'finishDate',
       'location', 'mode', 'name', 'organiserUrl', 'slug', 'startDate',
       'stewards', 'surfaces', 'templateFilename', 'timeZoneId',
       'timeZoneName', 'timeZoneOffset', 'trackingEventId']

In [651]:
q='''
SELECT o.*, t.totalDistance FROM (SELECT ce.`country.name`, ce.startDate, ce.finishDate, ce.name, ce.`country.iso3`, ce.surfaces,
COUNT(*) numOfStages, SUM(itc.distance) AS compDistanceKm FROM itinerary_controls itc
JOIN championship_events ce ON itc.eventId=ce.eventId
WHERE ce.`country.name`="{rally}" AND type='StageStart') AS o
JOIN (SELECT ce.`country.name`, SUM(itc.distance) totalDistance FROM itinerary_controls itc
JOIN championship_events ce ON itc.eventId=ce.eventId
WHERE ce.`country.name`="{rally}") AS t ON o.`country.name` = t.`country.name`
'''.format(rally=rally)
pd.read_sql(q,conn)

,country.name,startDate,finishDate,name,country.iso3,surfaces,numOfStages,compDistanceKm,totalDistance
0,Sweden,2018-02-15,2018-02-18,Rally Sweden,SWE,"Snow, Ice",19,314.25,1400.79


In [604]:
q='''
SELECT il.name AS date, itc.*,
    isc.itineraryLegId, isc.itinerarySectionId, isc.name AS section, isc.`order`
FROM itinerary_controls itc
JOIN championship_events ce ON itc.eventId=ce.eventId
JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
'''.format(rally=rally)
time_controls = pd.read_sql(q,conn)
time_controls['firstCarDueDateTimeLocal']=pd.to_datetime(time_controls['firstCarDueDateTimeLocal'])
time_controls.head()

,date,code,controlId,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,...,targetDurationMs,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,itineraryLegId,itinerarySectionId,section,order,surfaces
0,Thursday 15 February,TC1,1249,NaN,27,2018-02-15T19:04:00,2018-02-15 19:04:00,Start Farjestad Trotting Course Karlstad,317,Completed,...,0,Minute,TimeControl,53,117,53,117,Section 1,1,"Snow, Ice"
1,Thursday 15 February,SS1,1251,1.90,27,2018-02-15T19:08:00,2018-02-15 19:08:00,Super Special Stage Karlstad 1,317,Completed,...,240000,Minute,StageStart,53,117,53,117,Section 1,1,"Snow, Ice"
2,Thursday 15 February,TC1A,1194,102.33,27,2018-02-15T20:54:00,2018-02-15 20:54:00,Parc Ferme Torsby IN,-1,Completed,...,6360000,Minute,TimeControl,53,117,53,117,Section 1,1,"Snow, Ice"
3,Friday 16 February,TC1B,1193,NaN,27,2018-02-16T05:00:00,2018-02-16 05:00:00,Parc Ferme Torsby OUT/Service IN,-1,Completed,...,0,Minute,TimeControl,54,118,54,118,Section 2,2,"Snow, Ice"
4,Friday 16 February,TC1C,1195,NaN,27,2018-02-16T05:15:00,2018-02-16 05:15:00,Service OUT,-1,Completed,...,900000,Minute,TimeControl,54,118,54,118,Section 2,2,"Snow, Ice"


In [627]:
rules = {'origin': "#firstCarDueDateTimeLocal.pdtime(%H:%M:%S)# #code# #location# #distance.isNotNull(post=km).brackets# \[#targetDuration#\]",
         'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'car':"#manufacturername# #vehicleModel#",
        'entrant':"#entrantname#"}

In [628]:
grouped = time_controls.groupby('date',sort=False)
for key in grouped.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=grouped.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        print('{}\n'.format(key2))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Thursday 15 February:

Section 1

	19:04:00 TC1 Start Farjestad Trotting Course Karlstad  [00:00:00]
	19:08:00 SS1 Super Special Stage Karlstad 1 (1.9km) [00:04:00]
	20:54:00 TC1A Parc Ferme Torsby IN (102.33km) [01:46:00]


---

Friday 16 February:

Section 2

	05:00:00 TC1B Parc Ferme Torsby OUT/Service IN  [00:00:00]
	05:15:00 TC1C Service OUT  [00:15:00]
	06:52:00 TC2 Hof-Finniskog 1 (92.67km) [01:37:00]
	06:55:00 SS2 Hof-Finniskog 1 (21.26km) [00:03:00]
	08:04:00 TC3 Svullrya 1 (44.69km) [01:09:00]
	08:07:00 SS3 Svullrya 1 (24.88km) [00:03:00]
	08:51:00 TC4 Röjden 1 (16.44km) [00:44:00]
	08:54:00 SS4 Röjden 1 (19.13km) [00:03:00]
	10:16:00 TC4A Regroup and Technical Zone IN (54.78km) [01:22:00]


Section 3

	10:31:00 TC4B Regroup OUT/Service IN  [00:15:00]
	11:01:00 TC4C Service OUT  [00:30:00]
	12:38:00 TC5 Hof-Finniskog 2 (92.67km) [01:37:00]
	12:41:00 SS5 Hof-Finniskog 2 (21.26km) [00:03:00]
	14:00:00 TC6 Svullrya 2 (44.69km) [01:19:00]
	14:03:00 SS6 Svullrya 2 (24.88km) [

## Example Stage Report

In stage SS1, J. LATVALA (FIN) and M. ANTTILA (FIN) in the Toyota Yaris WRC took the stage, and the lead of the rally, with a time of 1:34.1, +0.6 ahead of second on stage T. NEUVILLE (BEL) and N. GILSOUL (BEL) (Hyundai Motorsport).

In [708]:
rc='RC2'
ss='SS4'
rally='Sweden'

In [777]:
rc='RC1'

#stage_times_overall, stage_times_stage
typ='stage_times_overall'
limit=3

q='''
SELECT st.*, sc.name as class, i.code, i.distance, i.name, sl.*
FROM {typ} st INNER JOIN itinerary_stages i ON st.stageid = i.stageid
INNER JOIN startlist_classes sc ON sc.entryid = st.entryid 
INNER JOIN championship_events ce ON i.eventId=ce.eventId
INNER JOIN startlists sl ON sl.entryId=sc.entryId
WHERE i.code="{ss}" 
    AND sc.name="{rc}" 
    AND ce.`country.name`="{rally}" 
ORDER BY position LIMIT {limit}

'''.format(rc=rc,ss=ss, rally=rally, typ=typ, limit=limit)
stageresult=pd.read_sql(q,conn)
stageresult

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,...,group.groupId,identifier,manufacturer.logoFilename,manufacturer.manufacturerId,manufacturer.name,manufacturerId,priority,status,tyreManufacturer,vehicleModel
0,PT0S,0,PT0S,0,1578,PT0S,0,1,PT35M49.7S,2149700,...,10,5,hyundai,33,Hyundai,33,P1,Entry,Michelin,i20 Coupe WRC
1,PT3.7S,3700,PT3.7S,3700,1577,PT0S,0,2,PT35M53.4S,2153400,...,10,4,hyundai,33,Hyundai,33,P1,Entry,Michelin,i20 Coupe WRC
2,PT5.3S,5300,PT1.6S,1600,1582,PT0S,0,3,PT35M55S,2155000,...,10,9,toyota,84,Toyota,84,P1,Entry,Michelin,Yaris WRC


In [780]:
cols=['driver.fullName','position', 'stageTime']
tmp = stageresult[cols]
tmp.columns = [c.replace('.','') for c in cols]
dd={'ss':'SS4'}
for l in tmp.to_dict(orient='records'):
    print(l)
    for k in l:
        dd['{}{}'.format(k,l['position'])] = l[k]
dd

{'driverfullName': 'Thierry NEUVILLE', 'position': 1, 'stageTime': 'PT35M49.7S'}
{'driverfullName': 'Andreas MIKKELSEN', 'position': 2, 'stageTime': 'PT35M53.4S'}
{'driverfullName': 'Esapekka LAPPI', 'position': 3, 'stageTime': 'PT35M55S'}


{'driverfullName1': 'Thierry NEUVILLE',
 'driverfullName2': 'Andreas MIKKELSEN',
 'driverfullName3': 'Esapekka LAPPI',
 'position1': 1,
 'position2': 2,
 'position3': 3,
 'ss': 'SS4',
 'stageTime1': 'PT35M49.7S',
 'stageTime2': 'PT35M53.4S',
 'stageTime3': 'PT35M55S'}

In [784]:
rules = {'origin': "In stage #ss#, #pos1#, with #pos2#,",
         'driver': "#driverfullName# (#drivercountryiso3#)",
                 'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'pos1': '#driverfullName1# was #position1.ordinal.number_to_words# in a time of #stageTime1#',
         'pos2': '#driverfullName2# in #position2.ordinal.number_to_words# in a time of #stageTime2#',
        }

for k in dd:
    rules[k]=str(dd[k])

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.add_modifiers(inflect_english)

grammar.flatten('#origin#')

'In stage SS4, Thierry NEUVILLE was first in a time of PT35M49.7S, with Andreas MIKKELSEN in second in a time of PT35M53.4S,'

# stuff..

In [609]:
q='SELECT st.*, sc.name as class, i.code, i.distance, i.name FROM split_times st INNER JOIN itinerary_stages i JOIN startlist_classes sc ON st.stageid = i.stageid AND sc.entryid = st.entryid WHERE code="SS4" and sc.name="RC1" ORDER BY entryId, splitPointTimeId'
pd.read_sql(q,conn).head(3)

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,class,code,distance,name
0,PT4M54.036S,294036,1483,2018-01-26T09:08:54.036Z,2018-01-26T10:08:54.036+01:00,585,13978,00:18:25.3000000,1105300.0,2018-01-26T09:04:00,2018-01-26T10:04:00+01:00,285,RC1,SS4,30.54,Roussieux - Eygalayes 1
1,PT8M7.632S,487632,1483,2018-01-26T09:12:07.632Z,2018-01-26T10:12:07.632+01:00,609,13999,00:18:25.3000000,1105300.0,2018-01-26T09:04:00,2018-01-26T10:04:00+01:00,285,RC1,SS4,30.54,Roussieux - Eygalayes 1
2,PT12M4.368S,724368,1483,2018-01-26T09:16:04.368Z,2018-01-26T10:16:04.368+01:00,608,14009,00:18:25.3000000,1105300.0,2018-01-26T09:04:00,2018-01-26T10:04:00+01:00,285,RC1,SS4,30.54,Roussieux - Eygalayes 1


In [116]:
q='SELECT * FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid'
dd=pd.read_sql(q,conn)
dd.head()

,codriver.abbvName,codriver.code,codriver.country.countryId,codriver.country.iso2,codriver.country.iso3,codriver.country.name,codriver.countryId,codriver.firstName,codriver.fullName,codriver.lastName,...,manufacturer.name,manufacturerId,priority,status,tyreManufacturer,vehicleModel,eventClassId,eventId,name,entryId
0,J. INGRASSIA,ING,76,FR,FRA,France,76,Julien,Julien INGRASSIA,INGRASSIA,...,Ford,26,P1,Entry,Michelin,FIESTA WRC,103,26,RC1,1483
1,D. BARRITT,BAR,235,GB,GBR,United Kingdom of Great Britain and Northern I...,235,Daniel,Daniel BARRITT,BARRITT,...,Ford,26,P1,Entry,Michelin,FIESTA WRC,103,26,RC1,1484
2,X. PANSERI,PAN,76,FR,FRA,France,76,Xavier,Xavier PANSERI,PANSERI,...,Ford,26,P1,Entry,Michelin,FIESTA WRC,103,26,RC1,1485
3,A. JAEGER,JAE,166,NO,NOR,Norway,166,Anders,Anders JAEGER,JAEGER,...,Hyundai,33,P1,Rejoined,Michelin,i20 WRC,103,26,RC1,1486
4,N. GILSOUL,GIL,22,BE,BEL,Belgium,22,Nicolas,Nicolas GILSOUL,GILSOUL,...,Hyundai,33,P1,Entry,Michelin,i20 WRC,103,26,RC1,1487


In [109]:
q='SELECT * FROM startlist_classes'
pd.read_sql(q,conn).head(3)

,eventClassId,eventId,name,entryId
0,103,26,RC1,1483
1,103,26,RC1,1484
2,103,26,RC1,1485


In [14]:
dd.columns

Index(['codriver.abbvName', 'codriver.code', 'codriver.country.countryId',
       'codriver.country.iso2', 'codriver.country.iso3',
       'codriver.country.name', 'codriver.countryId', 'codriver.firstName',
       'codriver.fullName', 'codriver.lastName', 'codriver.personId',
       'codriverId', 'driver.abbvName', 'driver.code',
       'driver.country.countryId', 'driver.country.iso2',
       'driver.country.iso3', 'driver.country.name', 'driver.countryId',
       'driver.firstName', 'driver.fullName', 'driver.lastName',
       'driver.personId', 'driverId', 'eligibility', 'entrant.entrantId',
       'entrant.logoFilename', 'entrant.name', 'entrantId', 'entryId',
       'eventId', 'group.name', 'groupId', 'group.groupId', 'identifier',
       'manufacturer.logoFilename', 'manufacturer.manufacturerId',
       'manufacturer.name', 'manufacturerId', 'priority', 'status',
       'tyreManufacturer', 'vehicleModel'],
      dtype='object')

In [120]:
dd.columns=[c.replace('.','') for c in dd.columns]

In [85]:
dd.head()

,codriverabbvName,codrivercode,codrivercountrycountryId,codrivercountryiso2,codrivercountryiso3,codrivercountryname,codrivercountryId,codriverfirstName,codriverfullName,codriverlastName,...,groupgroupId,identifier,manufacturerlogoFilename,manufacturermanufacturerId,manufacturername,manufacturerId,priority,status,tyreManufacturer,vehicleModel
0,J. INGRASSIA,ING,76,FR,FRA,France,76,Julien,Julien INGRASSIA,INGRASSIA,...,10,1,ford,26,Ford,26,P1,Entry,Michelin,FIESTA WRC
1,D. BARRITT,BAR,235,GB,GBR,United Kingdom of Great Britain and Northern I...,235,Daniel,Daniel BARRITT,BARRITT,...,10,2,ford,26,Ford,26,P1,Entry,Michelin,FIESTA WRC
2,X. PANSERI,PAN,76,FR,FRA,France,76,Xavier,Xavier PANSERI,PANSERI,...,10,3,ford,26,Ford,26,P1,Entry,Michelin,FIESTA WRC
3,A. JAEGER,JAE,166,NO,NOR,Norway,166,Anders,Anders JAEGER,JAEGER,...,10,4,hyundai,33,Hyundai,33,P1,Rejoined,Michelin,i20 WRC
4,N. GILSOUL,GIL,22,BE,BEL,Belgium,22,Nicolas,Nicolas GILSOUL,GILSOUL,...,10,5,hyundai,33,Hyundai,33,P1,Entry,Michelin,i20 WRC


In [693]:
rules = {'origin': "#driver# and #codriver# #ffo# #entrant#.",
         'ffo':['for the', 'from the', 'of the'],
         'driver': "#driverfullName# (#drivercountryiso3#)",
         'codriver':[ "#codriverfullName# (#codrivercountryiso3#)", "co-driver #codriverfullName# (#codrivercountryiso3#)"],
         'car':"#manufacturername# #vehicleModel#",
        'entrant':"#entrantname#"}

In [694]:
dd['report'] = dd.apply(lambda row: pandas_row_mapper(row, rules, "#origin#"), axis=1)


In [695]:
for i,r in dd.head().iterrows():
    print(r['report'])

Sébastien OGIER (FRA) and co-driver Julien INGRASSIA (FRA) from the M-SPORT FORD WORLD RALLY TEAM.
Elfyn EVANS (GBR) and co-driver Daniel BARRITT (GBR) from the M-SPORT FORD WORLD RALLY TEAM.
Bryan BOUFFIER (FRA) and co-driver Xavier PANSERI (FRA) for the M-SPORT FORD WORLD RALLY TEAM.
Andreas MIKKELSEN (NOR) and co-driver Anders JAEGER (NOR) of the HYUNDAI SHELL MOBIS WRT.
Thierry NEUVILLE (BEL) and co-driver Nicolas GILSOUL (BEL) from the HYUNDAI SHELL MOBIS WRT.


In [696]:
grouped = dd.groupby('name')

for name, group in grouped:
    print('Group {}:'.format(name))
    for i,r in group.head().iterrows():
        print(r['report'])

Group RC1:
Sébastien OGIER (FRA) and co-driver Julien INGRASSIA (FRA) from the M-SPORT FORD WORLD RALLY TEAM.
Elfyn EVANS (GBR) and co-driver Daniel BARRITT (GBR) from the M-SPORT FORD WORLD RALLY TEAM.
Bryan BOUFFIER (FRA) and co-driver Xavier PANSERI (FRA) for the M-SPORT FORD WORLD RALLY TEAM.
Andreas MIKKELSEN (NOR) and co-driver Anders JAEGER (NOR) of the HYUNDAI SHELL MOBIS WRT.
Thierry NEUVILLE (BEL) and co-driver Nicolas GILSOUL (BEL) from the HYUNDAI SHELL MOBIS WRT.
Group RC2:
Eric CAMILLI (FRA) and co-driver Benjamin VEILLAS (FRA) from the M-SPORT WORLD RALLY TEAM.
Jan KOPECKY (CZE) and co-driver Pavel DRESLER (CZE) of the SKODA MOTORSPORT II.
Teemu SUNINEN (FIN) and co-driver Mikko MARKKULA (FIN) for the M-SPORT WORLD RALLY TEAM.
Kevin ABBRING (NLD) and Pieter TSJOEN (BEL) of the ABBRING Kevin.
Guillaume DE MEVIUS (BEL) and Louis LOUKA (BEL) from the DE MEVIUS Guillaume.
Group RC3:
Ismaël VUISTINER (CHE) and Florine KUMMER (CHE) from the VUISTINER Ismaël.
Philippe RAGEAU (F

In [131]:
#itinerary_event, itinerary_legs, itinerary_sections, itinerary_stages, itinerary_controls

In [2]:
import sys
sys.executable

'/Users/ajh59/anaconda3/bin/python'

In [1]:
import tracery
from tracery.modifiers import base_english

import inflect
p = inflect.engine()

#conj ,sep ,final_sep , sep_spaced ,conj_spaced
#p.join(['as','df', 'er'], sep=',',final_sep='' )

In [5]:
#!/Users/ajh59/anaconda3/bin/pip install git+https://github.com/pwdyson/inflect.py

#----
#inflect.py modifiers
#Mappings based on https://github.com/pwdyson/inflect.py

from pandas import isnull
import time

import inflect
p = inflect.engine()


def ordinal(text, *params):
    return p.ordinal(text)

def number_to_words(text, *params):
    kwargs=dict(param.split('=') for param in params)
    try:
        resp = p.number_to_words(text, **kwargs)
    except:
        resp = "no recorded"
    return resp
    

def plural(text, *params):
    if params: return p.plural(text, params[0])
    return p.plural(text)

def plural_noun(text, *params):
    if params: return p.plural_noun(text, params[0])
    return p.plural_noun(text)

def plural_verb(text, *params):
    if params: return p.plural_verb(text, params[0])
    return p.plural_verb(text)

def plural_adj(text, *params):
    if len(params): return p.plural_adj(text, params[0])
    return p.plural_adj(text)
    
def singular_noun(text, *params):
    return p.singular_noun(text)
    
def no(text, *params):
    return p.no(text)
    
def num(text, *params):
    return p.num(text)
    
def compare(text, *params):
    return p.compare(text, params[0])
    
def compare_nouns(text, *params):
    return p.compare_nouns(text,params[0])

def compare_adjs(text, *params):
    return p.compare_adjs(text,params[0])

def a(text, *params):
    return p.a(text)

def an(text, *params):
    return p.an(text)

def present_participle(text, *params):
    return p.present_participle(text)

def classical(text, *params):
    kwargs=dict(param.split('=') for param in params)
    return p.classical(text, **kwargs)
    
def gender(text, *params):
    return p.gender(text)
    
def defnoun(text, *params):
    return p.defnoun(text)
    
def defverb(text, *params):
    return p.defverb(text)
    
def defadj(text, *params):
    return p.defadj(text)
    
def defa(text, *params):
    return p.defa(text)

def defan(text, *params):
    return p.defan(text)
    
def istrue(flag, *params):
    if flag=='True': return params[0]
    else: return ''
    
def isNotNull(text, *params):
    if isnull(text) or text=='nan': return ''
    kwargs=dict(param.split('=') for param in params)
    pre = kwargs['pre'] if 'pre' in kwargs else ''
    post = kwargs['post'] if 'post' in kwargs else ''
    brackets = True if 'brackets' in kwargs else False
    txt = "({pre}{text}{post})" if brackets else "{pre}{text}{post}"
    return txt.format(text=text,pre=pre,post=post)
    
def ifelse(flag, *params):
    if flag=='True': return params[0]
    else: return params[1]

def int_to_words(text, *params):
    try:
        text=int(float(text))
    except: pass
    kwargs=dict(param.split('=') for param in params)
    return p.number_to_words(text, **kwargs)

def pdtime(text, *params):
    if params:
        return pd.to_datetime(text).strftime(','.join(params))
    return pd.to_datetime(text).strftime("%b %d %Y %H:%M:%S")

def brackets(text, *params):
    if text:
        return '({})'.format(text)
    else: return ''
    

#--

inflect_english = {
    'compare':compare,
    'compare_nouns':compare_nouns,
    'compare_adjs':compare_adjs,
    'a':a,
    'a2':a,
    'an':an,
    'ordinal': ordinal,
    'number_to_words': number_to_words, 
    'plural':plural,
    'plural_noun':plural_noun,
    'plural_verb':plural_verb,
    'plural_adj':plural_adj,
    'singular_noun':singular_noun,
    'no':no,
    'num':num,
    'present_participle':present_participle,
    'classical':classical,
    'gender':gender,
    'defnoun':defnoun,
    'defverb':defverb,
    'defadj':defadj,
    'defa':defa,
    'defan':defan,
    #
    'int_to_words': int_to_words, 
    'istrue': istrue,
    'ifelse': ifelse,
    'isNotNull':isNotNull,
    'pdtime': pdtime,
    'brackets':brackets,
}


def inc(text, *params):
    return str(int(text)+1)

def dec(text, *params):
    return str(int(text)-1)

def branch(text, *params):
    if text==params[0].strip(): 
        ans = params[1].strip()
    else:
        ans = params[2].strip()
    return '[condition:#{}#]'.format(ans)

def whileNot0(text, *params):
    if int(text) > 0:
        return '[i:{i}]'.format(i=str(int(text)-1))
    return '[do:#{}#]'.format(params[0].strip())

pytracery_logic = {
    'inc': inc,
    'dec': dec,
    'branch':branch,
    'whileNot0': whileNot0
}



In [6]:
rules = {'origin': 'Loop test: \n#loop#',
         'loop':'[while:#i.whileNot0(end)#]#while##do#',
         'do':'#action##loop#',
         'action':'- count is at #i#\n',
         'end':'All done'
        }

#something like: here we go: one, two, three. Done
grammar = tracery.Grammar(rules)
grammar.add_modifiers(pytracery_logic)
print(grammar.flatten("[i:6]#origin#"))

Loop test: 
- count is at 5
- count is at 4
- count is at 3
- count is at 2
- count is at 1
- count is at 0
All done


In [7]:
rules={'origin':'Logical test:[branch:#testcondition#]#branch# Testing #value# gives: #condition#',
       'testcondition': '#value.branch(VeryTrue, answer1, answer2)#',
    'answer1':'This is true...',
    'answer2':'This is false...',
    'answer3':'nope',
    'condition':'answer3'
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(pytracery_logic)
print(grammar.flatten("[value:VeryTrue]#origin#"))
print(grammar.flatten("[value:notTrue]#origin#"))

Logical test: Testing VeryTrue gives: This is true...
Logical test: Testing notTrue gives: This is false...


In [8]:
rules = {'origin': 'Count demo - #countdemo#',
         'countdemo':'Count at [cnt:#initval#]#cnt#.\n#addone#, #addanother#, #takeone#',
         'addone': 'Add one gives [cnt:#cnt.inc#]#cnt#',
         'addanother': 'add another one gives [cnt:#cnt.inc#]#cnt#',
         'takeone':'take one gives [cnt:#cnt.dec#]#cnt.int_to_words#',
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(pytracery_logic)
grammar.add_modifiers(inflect_english)

print(grammar.flatten('[initval:1]#origin#'))

Count demo - Count at 1.
Add one gives 2, add another one gives 3, take one gives two


In [9]:
rules = {
    'origin': ['[mylocation:#location#]#origin2#'],
    'origin2': '#hello#  #hello.capitalize#, #location.plural(1)#! #mylocation# #bin.istrue(this is the answer)# #bin.ifelse(ans1,ans2)# #num.number_to_words(decimal=mark,andword=annnnd).uppercase#',
    '1':'a',
    '2':'b',
    'init':'1',
    'counter': '[cnt:#init#]',
    'cntdemo':'#counter# [cnt:#cnt.inc#] Now at: #cnt#, [cnt:#cnt.inc#]#cnt#, [cnt:#cnt.dec#]#cnt.int_to_words#',
    'hello': ['hello', 'greetings', 'howdy', 'hey','#asd#'],
    'asd': 'ASD',
    'bin':[ 'True'],
    'ans1':'a1',
    'ans2':['a2 2a'],
    'location': ['world', 'solar system', 'galaxy', 'universe'],
    'num': ['1023.5','12.12', '203']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.add_modifiers(inflect_english)
print(grammar.flatten("#origin#"))
print(grammar.flatten("#cntdemo#"))

howdy  Hello, world! solar system this is the answer ans1 ONE THOUSAND ANNNND TWENTY-THREE MARK FIVE
  Now at: 1((.inc)), 1((.inc))((.inc)), one point zero


In [10]:
import shlex

line='This is, a line, "with a comma, ok?"'
shlex.split(line)

['This', 'is,', 'a', 'line,', 'with a comma, ok?']

In [11]:
line.split(',')

['This is', ' a line', ' "with a comma', ' ok?"']

In [12]:
rules={
    'origin': '''here's my sentence: #test.uppercase.echo(this is, my sentence, "with a comma, ok?")#''',
    'test': 'Ping',
    'pushtest':'#test1#, #test2#',
    'test1':"[pet:#animal#]You have a #pet#. [pet:#animal#]Pet:#pet# ",
    'test2':"This: [pet:POP]Pet:#pet#",
    'animal':["dog",'cat'],
    'nest':'#[#test1#]pet#' #set some values
}

def myecho(*params):
    return '::'.join(params)

grammar_test= {
    'echo': myecho
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.add_modifiers(grammar_test)
print(grammar.flatten("#origin#"))


print(grammar.flatten("#nest#"))

here's my sentence: PING::this is:: my sentence:: "with a comma:: ok?"
dog


In [51]:
import tracery
from tracery.modifiers import base_english

from pandas import isnull
import time


import inflect
p = inflect.engine()


def inc(text, *params):
    return str(int(text)+1)
 
def dec(text, *params):
    return str(int(text)-1)
 
def branch(text, *params):
    if text==params[0].strip(): 
        ans = params[1].strip()
    else:
        ans = params[2].strip()
    return '[answer:#{}#]'.format(ans)
 
def whileNot0(text, *params):
    if int(text) > 0:
        return '[i:{i}]'.format(i=str(int(text)-1))
    return '[do:#{}#]'.format(params[0].strip())

def takeFrom(text, *params):
    return str(int(params[0].strip()) - int(text))

def add(text, *params):
    return str(int(text) + int(params[0].strip()))

def subtract(text, *params):
    return str(int(text) - int(params[0].strip()))

def multiply(text, *params):
    return str(int(text) * int(params[0].strip()))

def divide(text, *params):
    return str(int(text) / int(params[0].strip()))
    
def neg(text, *params):
    return str(-int(text))

def subvalue(text, *params):
    return '[i:#{x}._neg._subtract(-{y})._neg#]'.format(x=params[0],y=str(text) )
 
pytracery_logic = {
    '_inc': inc,
    '_dec': dec,
    '_branch':branch,
    '_whileNot0': whileNot0,
    '_takeFrom':takeFrom,
    '_neg':neg, 
    '_add':add,
    '_subtract':subtract,
    '_multiply':multiply,
    '_divide':divide,
    '_subvalue':subvalue
}

In [58]:
rules = {'origin': '[i:1]Start with: #i# #add1#, #subtract2#, #multiply3#, #divide4#',
         'add1':'[i:#i._add(10)#]#i#\n',
         'subtract2':'[i:#i._subtract(2)#]#i#\n',
         'multiply3':'[i:#i._multiply(3)#]#i#\n',
         'divide4':'[i:#i._divide(4)#]#i#\n',
         'subto':'[i:10][j:2][i:#i._subvalue(j)#] ss #j._neg._subtract(-2)# is #i# ok'
        }
 
grammar = tracery.Grammar(rules)
grammar.add_modifiers(pytracery_logic)
grammar.add_modifiers(inflect_english)

print(grammar.flatten("#origin#"))

print(grammar.flatten("#subto#"))

Start with: 1 11
, 9
, 27
, 6.75

 ss 0 is  ok


In [15]:
rules = {'origin': 'Loop test: \n#loop#',
         'loop':'[while:#i._whileNot0(end)#]#while##do#',
         'do':'#action##loop#',
         'action':'[j:#j._inc#]- for the #j.number_to_words.ordinal# time of asking...\n',
         'end':'All done'
        }
 
grammar = tracery.Grammar(rules)
grammar.add_modifiers(pytracery_logic)
grammar.add_modifiers(inflect_english)

print(grammar.flatten("[j:-3][i:#j._neg#][j:0]#origin#"))
#[cnt:#cnt.inc#]#cnt#

Loop test: 
- for the first time of asking...
- for the second time of asking...
- for the third time of asking...
All done


In [16]:
rules = {'origin': '#test# #test# #test#',
         'test':["as","df"]
        }
 
grammar = tracery.Grammar(rules)
grammar.add_modifiers(pytracery_logic)
grammar.add_modifiers(inflect_english)

print(grammar.flatten("#origin#"))

as df as
